In [126]:
import bokeh.plotting as bk
from bokeh.layouts import Row, WidgetBox, Column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Paragraph, TextInput, Select, RadioGroup
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

import numpy as np
import scipy.signal as signal

bk.output_notebook()

Loading BokehJS ...

In [162]:
class FilterDesignTool():
    
    def __init__(self,
                 width=800,
                 height=600):
        
        self.source = ColumnDataSource({
            'x0': [],
            'y0': [],
            'x1': [],
            'y1': [],
            'x2': [],
            'y2': [],
            'x3': [],
            'y3': [],
            'x4': [],
            'y4': [],
            'x5': [],
            'y5': [],
            'x6': [],
            'y6': []
        })
        
        self.filt_source = ColumnDataSource({'nyquist': [],
                                             'wp': [],
                                             'ws': [],
                                             'gstop': [],
                                             'gpass': [],
                                             'b': [],
                                             'a': []
                                            })
        self.filt_image_source = ColumnDataSource({
            'w': [],
            'h': []
        })
        
        self.filter_figure = bk.figure(width=width, height=height)
        self.init_controls()
        self.update_parameters()
        self.init_plot()
        self.update_filter()
        self.plot_filter()
        
        filter_information_widgets = WidgetBox(Paragraph(text="Filter Information"))
        response_type_widgets = WidgetBox(Paragraph(text="Response Type"),
                                          self.response_type)
        design_method_widgets = WidgetBox(Paragraph(text="Design Method"),
                                          self.design_method, self.design_method_select)
        filter_order_widgets = WidgetBox(Paragraph(text="Filter Order"),
                                         self.specify_order,
                                         self.specify_order_input)
        options_widgets = WidgetBox(Paragraph(text="Options"),
                                    self.density_factor)
        frequency_specifications_widgets = WidgetBox(Paragraph(text="Frequency Specifications"),
                                                     self.freq_units, self.fs, self.fpass, self.fstop)
        magnitude_specifications_widgets = WidgetBox(Paragraph(text="Magnitude Specifications"),
                                                     self.mag_units, self.amp_pass, self.amp_stop)
        
        upper_row = Row(filter_information_widgets,
                        self.filter_figure)
        lower_row = Row(Column(response_type_widgets, design_method_widgets),
                        Column(filter_order_widgets, options_widgets),
                        frequency_specifications_widgets,
                        magnitude_specifications_widgets)
        
        self.layout = Column(upper_row, lower_row)
        
    def init_controls(self):
        
        self.response_type = RadioGroup(labels=["Lowpass",
                                        "Highpass",
                                        "Bandpass",
                                        "BandStop",
                                        "Differentiator"],
                                        active=0)
        self.response_type.on_change('active', self.update)
        
        self.design_method = RadioGroup(labels=["IIR", "FIR"], active=0)
        self.design_method_select = Select(options=["Butterworth"],
                                           value="Butterworth",
                                           title="Method:")
        
        self.specify_order = RadioGroup(labels=["Min Order", "Specify Order"], active=0)
        self.specify_order_input = TextInput(value='10')
        
        self.density_factor = TextInput(value="20", title="Density Factor")
        
        self.freq_units = Select(value='Hz', title='Units:', options=['Hz', 'Normalised (0-1)'])
        self.fs = TextInput(value='1000.0', title='Fs:')
        self.fs.on_change('value', self.update)
        self.fpass = TextInput(value='100.0', title='Fpass:')
        self.fstop = TextInput(value='250.0', title='Fstop:')
        
        self.mag_units = Select(value='dB', title='Magnitude Units:', options=['dB'])
        self.amp_pass = TextInput(value='1.0', title='Apass:')
        self.amp_stop = TextInput(value='80.0', title='Astop:')
        
        controls = [self.design_method_select,
                    self.specify_order_input,
                    self.density_factor,
                    self.freq_units,
                    self.fs,
                    self.fpass,
                    self.fstop,
                    self.mag_units,
                    self.amp_pass,
                    self.amp_stop
                    ]
        radio_controls = [self.response_type,
                          self.design_method,
                          self.specify_order]
        
        [n.on_change('value', self.update) for n in controls]
        [n.on_change('active', self.update) for n in radio_controls]
        
    def update_parameters(self):
        
        x = {
            'x0': [float(self.fs.value)/2, float(self.fs.value)/2],
            'y0': [float(self.amp_stop.value)* -1.0, 0],
            'x1': [float(self.fstop.value), float(self.fstop.value)],
            'y1': [float(self.amp_stop.value)* -1.0, float(self.amp_pass.value) * -0.5],
            'x2': [float(self.fpass.value), float(self.fpass.value)],
            'y2': [float(self.amp_stop.value)* -1.0, float(self.amp_pass.value) * -0.5],
            'x3': [0, 0],
            'y3': [float(self.amp_stop.value)*-1.0, float(self.amp_pass.value) * -0.5],
            'x4': [0, float(self.fpass.value)],
            'y4': [float(self.amp_pass.value) * -1.0, float(self.amp_pass.value) * -1.0],
            'x5': [float(self.fstop.value), float(self.fs.value)/2],
            'y5': [float(self.amp_stop.value)* -1.0, float(self.amp_stop.value) * -1.0],
            'x6': [0, float(self.fpass.value)],
            'y6': [float(self.amp_pass.value) * 0.0, float(self.amp_pass.value) * 0.0]
        }
        
        self.source.data = x
        
    def init_plot(self):
        
        self.filter_figure.line('x0', 'y0', source=self.source)
        self.filter_figure.line('x1', 'y1', source=self.source)
        self.filter_figure.line('x2', 'y2', source=self.source)
        self.filter_figure.line('x1', 'y3', source=self.source)
        self.filter_figure.line('x4', 'y4', source=self.source)
        self.filter_figure.line('x5', 'y5', source=self.source)
        self.filter_figure.line('x6', 'y6', source=self.source)
        
        self.filter_figure.xaxis.axis_label = 'Frequency {0}'.format(self.freq_units.value)
        self.filter_figure.yaxis.axis_label = 'Amplitude {0}'.format(self.mag_units.value)
            
    def update_filter(self):
        
        if self.freq_units.value == 'Hz':
            x = {'nyquist': [float(self.fs.value) / 2],
                 'wp': [[float(wp) / (0.5*float(self.fs.value)) for wp in self.fpass.value.split(',')]],
                 'ws': [[float(ws) / (0.5*float(self.fs.value)) for ws in self.fstop.value.split(',')]],
                 'gstop': [float(self.amp_stop.value)],
                 'gpass': [float(self.amp_pass.value)],
                 'b': [],
                 'a': []}
            b, a = signal.iirdesign(wp=x['wp'],
                                      ws=x['ws'],
                                      gstop=x['gstop'][0],
                                      gpass=x['gpass'][0],
                                      ftype='ellip')
            x['b'] = [b]
            x['a'] = [a]
            w, h = signal.freqz(b, a)
            w_norm = x['nyquist'][0] * w / max(w)
            h_db = 20 * np.log10(abs(h))
            
            self.filt_image_source.data = {'w': w_norm,
                                           'h': h_db}
            
            self.filt_source.data = x
        
    def plot_filter(self):
        self.filter_figure.line('w',
                                'h',
                                source=self.filt_image_source,
                                line_color='black')
        
    def update(self, old, new, attr):
        self.update_parameters()
        self.update_filter()
    
    def modify_doc(self, doc):
        doc.add_root(self.layout)
    

In [163]:
p = FilterDesignTool()
handler = FunctionHandler(p.modify_doc)
app = Application(handler)

bk.show(app)

In [157]:
p.filt_source.data

{'nyquist': [500.0],
 'wp': [[0.2]],
 'ws': [[0.5]],
 'gstop': [90.0],
 'gpass': [1.0],
 'b': [array([0.00041319, 0.00027119, 0.00080036, 0.00059178, 0.00080036,
         0.00027119, 0.00041319])],
 'a': [array([  1.        ,  -4.85238436,  10.33042582, -12.27639538,
           8.56382154,  -3.32085817,   0.55938637])],
 'w': [array([  0.        ,   0.97847358,   1.95694716,   2.93542074,
           3.91389432,   4.89236791,   5.87084149,   6.84931507,
           7.82778865,   8.80626223,   9.78473581,  10.76320939,
          11.74168297,  12.72015656,  13.69863014,  14.67710372,
          15.6555773 ,  16.63405088,  17.61252446,  18.59099804,
          19.56947162,  20.54794521,  21.52641879,  22.50489237,
          23.48336595,  24.46183953,  25.44031311,  26.41878669,
          27.39726027,  28.37573386,  29.35420744,  30.33268102,
          31.3111546 ,  32.28962818,  33.26810176,  34.24657534,
          35.22504892,  36.2035225 ,  37.18199609,  38.16046967,
          39.13894325, 

In [141]:
signal.iirdesign?

Signature: signal.iirdesign(wp, ws, gpass, gstop, analog=False, ftype='ellip', output='ba')
Docstring:
Complete IIR digital and analog filter design.

Given passband and stopband frequencies and gains, construct an analog or
digital IIR filter of minimum order for a given basic type.  Return the
output in numerator, denominator ('ba'), pole-zero ('zpk') or second order
sections ('sos') form.

Parameters
----------
wp, ws : float
    Passband and stopband edge frequencies.
    For digital filters, these are normalized from 0 to 1, where 1 is the
    Nyquist frequency, pi radians/sample.  (`wp` and `ws` are thus in
    half-cycles / sample.)  For example:

        - Lowpass:   wp = 0.2,          ws = 0.3
        - Highpass:  wp = 0.3,          ws = 0.2
        - Bandpass:  wp = [0.2, 0.5],   ws = [0.1, 0.6]
        - Bandstop:  wp = [0.1, 0.6],   ws = [0.2, 0.5]

    For analog filters, `wp` and `ws` are angular frequencies (e.g. rad/s).

gpass : float
    The maximum loss in the passban

In [132]:
float(p.fs.value) * p.w / max(p.w)

array([   0.        ,    1.95694716,    3.91389432,    5.87084149,
          7.82778865,    9.78473581,   11.74168297,   13.69863014,
         15.6555773 ,   17.61252446,   19.56947162,   21.52641879,
         23.48336595,   25.44031311,   27.39726027,   29.35420744,
         31.3111546 ,   33.26810176,   35.22504892,   37.18199609,
         39.13894325,   41.09589041,   43.05283757,   45.00978474,
         46.9667319 ,   48.92367906,   50.88062622,   52.83757339,
         54.79452055,   56.75146771,   58.70841487,   60.66536204,
         62.6223092 ,   64.57925636,   66.53620352,   68.49315068,
         70.45009785,   72.40704501,   74.36399217,   76.32093933,
         78.2778865 ,   80.23483366,   82.19178082,   84.14872798,
         86.10567515,   88.06262231,   90.01956947,   91.97651663,
         93.9334638 ,   95.89041096,   97.84735812,   99.80430528,
        101.76125245,  103.71819961,  105.67514677,  107.63209393,
        109.5890411 ,  111.54598826,  113.50293542,  115.45988